# Game playing systems. Implementation

It is recommended to run the script in Google Collab, since in the last part of the script inputs are implemented to play with the algorithm and they are not very well handled in Visual Code Studio, for example.


In [ ]:
#libraries
import numpy as np
from copy import deepcopy
import random
import time

## Auxiliar functions

In [ ]:
def check_result(layout):
    """
    This function checks the state/result of the game given a layout defined as a np array.
    it returns:
        - 0 if it is a draw
        - 1 if it is a win 
        - -1 if it is a lose
    """
    dim = layout.shape[0]
    check = layout.sum(0) #rows
    check = np.append(check, layout.sum(1)) #cols
    check = np.append(check, sum(layout.diagonal())) # left-right diagonal
    check = np.append(check, sum(np.fliplr(layout).diagonal())) #right-left diagonal

    result = 0
    if dim in check: #Win
        result = 1
    elif -dim in check: #lose
        result = -1
    else:
        pass

    return result
    

## Node and tree logic
Below you can find the logic behind the nodes of the tree

In [ ]:
class Node:

    def __init__(self,parent, layout, player):

        # Storing the parent of the node for back propagating
        self.parent = parent

        # information of the layout and the player
        self.layout = layout
        self.player = player

        # Statistics of the node
        self.stats = Node_stats()
        
        # to control whether it has been visited
        self.visited = False

        #stores the result of the layout. 0 draw, 1 win, -1 lose
        self.result = check_result(self.layout)

        # if the layout still has 0 (to fill positions) or the result is different than a draw
        self.is_terminal =  not np.any(layout==0) or self.result !=0

        # Stores the children of the node, when created. If it is the root node
        # (does not have parents), create children
        self.children = []
        if self.parent == []:
          self.create_children()

    def create_children(self):
        """
        This function creates the children of a given node.
        """

        # If the node is not terminal and it does not have created children already
        if not self.is_terminal and self.children ==[]:

          #getting all coordinates that still have a 0 (haven't been played)
          coord = np.where((layout == 0))

          #for all coordinates that still have a 0 (haven't been played)
          for x, y in zip(coord[0], coord[1]):

              # Getting coordinates to fill children nodes with a possible play
              aux_layout = deepcopy(self.layout)
              aux_layout[x,y] = - self.player

              #Creating the node
              node = Node(self, aux_layout, - self.player)

              # appending to the children list
              self.children.append(node)
        
        return 

    def is_fully_expanded(self):
        """
        Checks if children nodes are visited.
        """

        fully_expanded = True if not self.is_terminal else False
        for child in self.children:
            fully_expanded = fully_expanded and child.visited

        return fully_expanded

    def UCT(self, N_parent, c= 0.2):
        """
        Calculates uct of the node, given the N of the parent node
        """
        return self.stats.Q/self.stats.N + c*np.sqrt(np.log(N_parent)/self.stats.N)

    def best_UCT_children(self):
        """
        Retrieves the children with the highest UCT
        """
        stats = []
        for child in self.children:
            stats.append(child.UCT(N_parent = self.stats.N))

        #retrieving the index of the children with the highest UCT    
        index_max = stats.index(max(stats))

        return self.children[index_max]

    def choose_unvisited_node(self):
        """
        Method to choose unvisited nodes when selecting in MCTS
        """

        candidates = []
        for child in self.children:
            if not child.visited:
                candidates.append(child)

        return random.choice(candidates)

    def choose_random_node(self):
        """
        Choose a random children node. Used in the rollout_policy
        """

        return random.choice(self.children)

    def favorite_child(self):
        """
        chooses the children with the highest number of visits
        """

        n_visitis = []
        for child in self.children:
            n_visitis.append(child.stats.N)

        #retrieving the index of the children with the highest UCT    
        index_max = n_visitis.index(max(n_visitis))

        return self.children[index_max]



class Node_stats:

    def __init__(self):
        self.N = 1
        self.Q = 0




## MonteCarlo Tree Search
The functions that implement the MonteCarlo Tree Search are defined below

In [ ]:
def selection(node):
    """
    Selection policy based on UCT
    """
    while node.is_fully_expanded():
        node = node.best_UCT_children()

    if not node.is_terminal:
        node = node.choose_unvisited_node()

    return node

def rollout(node):
    """
    Executes the simulation of the MCTS. Its policy is defined in roll_out_policy
    """
    # We set the node to visited and we create its children
    node.visited = True
    node.create_children()

    # Till a terminal state is found, keep going
    while not node.is_terminal:
        node = roll_out_policy(node)
        node.create_children()

    return node.result

def roll_out_policy(node):
    """
    Implements the logic of the simulation. In our case it is a random policy
    """

    return node.choose_random_node()

def backpropagate(node, result):
    """
    Implements the logic behind the learning and propagation of simulations
    """
    while node.parent != []:

        node.stats.N +=1
        node.stats.Q += result 

        node = node.parent


def MonteCarlo_TreeSearch(node, max_time = 5):
    """
    Implements the MCTS.
    It returns a selected node based on the number of visits that that node has.
    (favorite_child)
    """
    #track the amount of searches that it performs
    search_counter = 0

    time_beginning = time.time()
    while (time.time() - time_beginning < max_time):
      selected_node = selection(node)
      result = rollout(selected_node)
      backpropagate(selected_node, result)

      search_counter +=1
    
    print("The MCTS has performed ", search_counter, " searches in ", time.time() - time_beginning, " seconds.")
    return node.favorite_child()
    

## Examples
* How does the algorithm start?

In [ ]:
# Configuration
layout = np.zeros((3,3))
player = 1
parent = []

starting_node = Node(parent=[], player=-1, layout=layout)
starting_node.layout

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [ ]:
# Performing search
selected_node = MonteCarlo_TreeSearch(starting_node)

The MCTS has performed  22314  searches in  5.000095844268799  seconds.


In [ ]:
# Decision making
selected_node.layout

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

* Does the algorithm know how to finish?

In [ ]:
# Configuration
layout = np.zeros((3,3))
layout[0,0] = 1
layout[1,1] = 1
layout[0,2] = -1
layout[1,2] = -1

player = 1
parent = []

starting_node = Node(parent=[], player=-1, layout=layout)

In [ ]:
# State of the board
starting_node.layout

array([[ 1.,  0., -1.],
       [ 0.,  1., -1.],
       [ 0.,  0.,  0.]])

In [ ]:
# Decision after the search
selected_node = MonteCarlo_TreeSearch(starting_node)
selected_node.layout

The MCTS has performed  224896  searches in  5.000008821487427  seconds.


array([[ 1.,  0., -1.],
       [ 0.,  1., -1.],
       [ 0.,  0.,  1.]])

## Programming TIC-TAC-TOE to play against a random oponent


In [ ]:
def play_random(layout, player):
  """
  Defines a player that plays random
  """
  node = Node(parent=[], player=player, layout=layout)
  node = node.choose_random_node()
  return node.layout
  

Here we try it with a 4x4 board


In [ ]:
player = -1 #1 if the MCTS starts -1 if opponent starts
layout = np.zeros((4,4))
node = Node

while check_result(layout) == 0 and np.any(layout==0):
  if player == 1:
    starting_node = Node(parent=[], player=-player, layout=layout)
    selected_node = MonteCarlo_TreeSearch(starting_node)
    layout = selected_node.layout

  else:
    layout = play_random(layout, -player)


  print("Player: ", player)
  print(layout)
  print()

  player = -player


result = check_result(layout)

if result == 1:
  print("The machine wins")

elif result == -1:
  print("The random player wins!")

else:
  print("It is a draw!")

Player:  -1
[[ 0.  0.  0. -1.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]

The MCTS has performed  217  searches in  5.009582042694092  seconds.
Player:  1
[[ 0.  0.  1. -1.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]

Player:  -1
[[ 0.  0.  1. -1.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]

The MCTS has performed  296  searches in  5.009297609329224  seconds.
Player:  1
[[ 0.  0.  1. -1.]
 [ 0. -1.  1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]

Player:  -1
[[ 0.  0.  1. -1.]
 [ 0. -1.  1.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  0.  0.]]

The MCTS has performed  32007  searches in  5.000099182128906  seconds.
Player:  1
[[ 0.  0.  1. -1.]
 [ 0. -1.  1.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  1.  0.]]

Player:  -1
[[ 0. -1.  1. -1.]
 [ 0. -1.  1.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  1.  0.]]

The MCTS has performed  135051  searches in  5.000012159347534  seconds.
Player:  1
[[ 0. -1.  1. -1.]
 [ 0. -1.  1.  0.]
 [ 0. -1.  1.  0.]
 [ 0.  0.  1.  0.]]


## Programming TIC-TAC-TOE to play against a human


In [ ]:
def input_human_decision(layout, player):
  """
  Function that handles the human input
  """

  print("Your turn! Choose the row and the column of your movement")
  row = int(input("Choose row [0,1,2]: "))
  column = int(input("Choose column [0,1,2]: "))

  if layout[row, column] != 0:
    print("That position is already covered. Choose another one.")
    layout = input_human_decision(layout, player)

  else: 
    layout[row, column] = player

  return layout


In [ ]:
player = 1 #1 if the MCTS starts -1 if opponent starts
layout = np.zeros((3,3))
node = Node

print("Game starts! You are player -1")
print(layout)

while check_result(layout) == 0 and np.any(layout==0):
  if player == 1:
    starting_node = Node(parent=[], player=-player, layout=layout)
    selected_node = MonteCarlo_TreeSearch(starting_node)
    layout = selected_node.layout

  else:
    layout = input_human_decision(layout, player)


  print("Player: ", player)
  print(layout)
  print()

  player = -player

result = check_result(layout)

if result == 1:
  print("The machine wins")

elif result == -1:
  print("YOU win!")

else:
  print("It is a draw!")

Game starts! You are player -1
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
The MCTS has performed  20060  searches in  5.0000996589660645  seconds.
Player:  1
[[0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]]

Your turn! Choose the row and the column of your movement
Choose row [0,1,2]: 1
Choose column [0,1,2]: 1
Player:  -1
[[ 0.  0.  1.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]

The MCTS has performed  57220  searches in  5.0000319480896  seconds.
Player:  1
[[ 0.  1.  1.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]

Your turn! Choose the row and the column of your movement
Choose row [0,1,2]: 0
Choose column [0,1,2]: 0
Player:  -1
[[-1.  1.  1.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]

The MCTS has performed  78184  searches in  5.000036239624023  seconds.
Player:  1
[[-1.  1.  1.]
 [ 0. -1.  1.]
 [ 0.  0.  0.]]

Your turn! Choose the row and the column of your movement
Choose row [0,1,2]: 2
Choose column [0,1,2]: 2
Player:  -1
[[-1.  1.  1.]
 [ 0. -1.  1.]
 [ 0.  0. -1.]]

YOU win!
